In [ ]:
# standard inputs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


sns.set(style='whitegrid')
#plt.style.use('seaborn-vibrant')


In [ ]:
# Loading the raw dataset
df = pd.read_csv('../data/raw/vgsales.csv')

In [ ]:
# Display the first 5 rows of the DataFrame
#print(df.head())

# Display column names and their data types
#print("\nColumn Data Types:")
#print(df.dtypes)

# Display the number of missing values per column
#print("\nMissing Values Per Column:")
#print(df.isnull().sum())

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       10015 non-null  object 
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


Column | Missing? | Notes
Name | Yes (2) | Minor — we can drop these rows
Platform | No | Good
Year_of_Release | Yes (269) | Might be useful — we can fill or drop later
Genre | Yes (2) | Small enough to drop
Publisher | Yes (54) | Might be okay to drop or replace with "Unknown"
Critic_Score, Critic_Count | ~51% missing | Not ideal, but we’ll explore
User_Score | Lots missing + weird object type | We'll clean/convert
User_Count | ~55% missing | Risky to use
Developer | ~40% missing | Could drop
Rating | ~40% missing | Could be useful (E, M, T, etc.) — we’ll decide

### Data Cleaning

In [ ]:
# Drop rows with missing critical values
df = df.dropna(subset=['Name', 'Genre', 'Year_of_Release'])

# Fill missing Publisher and Rating with "Unknown"
df['Publisher'] = df['Publisher'].fillna('Unknown')
df['Rating'] = df['Rating'].fillna('Unknown')

# Convert Year to int
df['Year_of_Release'] = df['Year_of_Release'].astype(int)

# Convert User_Score to numeric (some are 'tbd')
df['User_Score'] = pd.to_numeric(df['User_Score'], errors='coerce')

# Drop columns that are too incomplete or not useful
df = df.drop(columns=['User_Count', 'Developer'])

# Check again
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 16448 entries, 0 to 16718
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16448 non-null  object 
 1   Platform         16448 non-null  object 
 2   Year_of_Release  16448 non-null  int64  
 3   Genre            16448 non-null  object 
 4   Publisher        16448 non-null  object 
 5   NA_Sales         16448 non-null  float64
 6   EU_Sales         16448 non-null  float64
 7   JP_Sales         16448 non-null  float64
 8   Other_Sales      16448 non-null  float64
 9   Global_Sales     16448 non-null  float64
 10  Critic_Score     7983 non-null   float64
 11  Critic_Count     7983 non-null   float64
 12  User_Score       7463 non-null   float64
 13  Rating           16448 non-null  object 
dtypes: float64(8), int64(1), object(5)
memory usage: 1.9+ MB


### HIT, AVERAGE, OR FLOP

In [ ]:
def label_success(sales):
    if sales > 1.0:
        return 'Hit'
    elif sales >= 0.1:
        return 'Average'
    else:
        return 'Flop'

# Create the new target column
df['Success_Level'] = df['Global_Sales'].apply(label_success)

# Check distribution
df['Success_Level'].value_counts()


Success_Level
Average    8625
Flop       5789
Hit        2034
Name: count, dtype: int64

### Selecting Feature for Modelling

In [ ]:
# Drop unnecessary columns
X = df[['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'Critic_Score', 'Critic_Count', 'User_Score', 'Rating']]
y = df['Success_Level']

<class 'pandas.core.frame.DataFrame'>
Index: 16448 entries, 0 to 16718
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Platform         16448 non-null  object 
 1   Year_of_Release  16448 non-null  int64  
 2   Genre            16448 non-null  object 
 3   Publisher        16448 non-null  object 
 4   Critic_Score     7983 non-null   float64
 5   Critic_Count     7983 non-null   float64
 6   User_Score       7463 non-null   float64
 7   Rating           16448 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 1.1+ MB


### Encode Categorical Variables
Convert text features into numerical format using One-Hot Encoding

In [ ]:
# One-Hot Encode the categorical features
X = pd.get_dummies(X, columns=['Platform', 'Genre', 'Publisher', 'Rating'], drop_first=True)

# Check the new feature set
X.head()

NameError: name 'pd' is not defined